In [ ]:
# class MyCompleter(object):  # Custom completer

    def __init__(self, options):
        self.options = sorted(options)

    def complete(self, text, k):
        if text:  # cache matches (entries that start with entered text)
            self.matches = [s for s in self.options 
                                if s and s.startswith(text)]
        else:  # no text entered, all matches possible
            self.matches = self.options[:]
        if len(self.matches) > k:
            self.matches = self.matches[:k]
        return self.matches
completer = MyCompleter(["hello", "hi", "how", "goodbye", "great"])
# the output is sorted in lexicographical order
# it could handle n-gram based input dictionary

In [2]:
print(completer.complete("h",2))

['hello', 'hi']
